In [1]:
import pandas as pd

# путь к сводной таблице
filepath = r"C:\Projects\geochem_python_lessons\Lessons\3 QAQC\DATA\geochem_results.xlsx"

# загрузим листы
results = pd.read_excel(filepath, sheet_name = 'Results')
metadata = pd.read_excel(filepath, sheet_name = 'Metadata', index_col = 0)
repeat_codes = pd.read_excel(filepath, sheet_name = 'Repeat_codes')

In [2]:
# удалим дубликаты анализов, оставив последний
results = results.drop_duplicates(subset = 'SampleID', keep = 'last')
results

,SampleID,Sr_pct_ПКСА,Ba_pct_ПКСА,TiO2_pct_ПКСА,MnO_pct_ПКСА,V_ppm_ПКСА,Cr_ppm_ПКСА,Co_ppm_ПКСА,Ni_ppm_ПКСА,Be_ppm_ПКСА,Li_ppm_ПКСА,W_ppm_ПКСА,Mo_ppm_ПКСА,Sn_ppm_ПКСА,Cu_ppm_ПКСА,Pb_ppm_ПКСА,Zn_ppm_ПКСА,Cd_ppm_ПКСА,Bi_ppm_ПКСА,Ag_ppm_ПКСА,As_ppm_ПКСА,Sb_ppm_ПКСА
0,101000,0.03,0.15,1.2,0.2,100,800,20,40,1.5,15,1.5,2.5,1.5,50,12,60,4,0.5,0.03,10,15
1,101001,0.02,0.1,1.2,0.2,100,300,20,30,1.5,30,1.5,3,2,100,200,200,4,0.5,0.4,10,15
2,101002,0.03,0.15,1.2,0.12,80,100,15,20,1.5,20,1.5,2,1.5,50,15,50,4,0.5,0.06,10,15
3,101003,0.05,0.025,1,0.15,150,150,25,40,1,30,1.5,1.5,1.5,80,12,60,4,0.5,0.04,10,15
4,101004,0.03,0.15,1.2,0.15,80,150,20,30,1.5,30,1.5,1.5,2,60,10,50,4,0.5,0.04,10,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12963,9301180,0.03,0.12,1.01,0.15,60,100,40,30,0.5,10,1.5,1.2,0.5,40,6,40,4,0.5,0.03,10,15
12964,9301178,0.05,0.08,1.01,0.2,60,20,20,8,1.5,5,1.5,3,0.5,30,8,40,4,0.5,0.03,10,15
12965,9301177,0.1,0.15,1.01,0.15,60,80,30,25,0.5,15,1.5,1.2,0.5,30,6,30,4,0.5,0.03,10,15
12966,9301175,0.03,0.1,1.01,0.1,80,100,40,50,0.5,25,1.5,1.5,1,120,8,60,4,0.5,0.04,10,15


In [3]:
# проверим таблицу метаданных
metadata

,Sr_pct_ПКСА,Ba_pct_ПКСА,TiO2_pct_ПКСА,MnO_pct_ПКСА,V_ppm_ПКСА,Cr_ppm_ПКСА,Co_ppm_ПКСА,Ni_ppm_ПКСА,Be_ppm_ПКСА,Li_ppm_ПКСА,W_ppm_ПКСА,Mo_ppm_ПКСА,Sn_ppm_ПКСА,Cu_ppm_ПКСА,Pb_ppm_ПКСА,Zn_ppm_ПКСА,Cd_ppm_ПКСА,Bi_ppm_ПКСА,Ag_ppm_ПКСА,As_ppm_ПКСА,Sb_ppm_ПКСА
Analyte Symbol,Sr,Ba,TiO2,MnO,V,Cr,Co,Ni,Be,Li,W,Mo,Sn,Cu,Pb,Zn,Cd,Bi,Ag,As,Sb
Unit Symbol,%,%,%,%,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm
Detection Limit,0.01,0.005,0.001,0.001,1,1,1,1,1,10,3,0.5,1,1,1,10,8,1,0.02,20,30
Analysis Method,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА
Sensitivity k,0.026491,0.000154,0.001313,0.000077,0,0,0.00085,0.000077,0.56835,0.200649,0.995907,0.000232,0.285449,0,0.000541,0.010658,0.995984,0.96177,0.026568,0.976908,0.999691


In [4]:
repeat_codes

,Repeat SampleID,Initial SampleID
0,К302195,302195
1,К302175,302175
2,К302171,302171
3,К301206,301206
4,К301183,301183
...,...,...
272,9301180,301180
273,9301178,301178
274,9301177,301177
275,9301175,301175


In [7]:
# проверим наличие пар анализов (первичный + повторный) в результатах
idx = repeat_codes['Repeat SampleID'].isin(results['SampleID']) & repeat_codes['Initial SampleID'].isin(results['SampleID'])
repeat_codes = repeat_codes.loc[idx, :]
repeat_codes

,Repeat SampleID,Initial SampleID
0,К302195,302195
1,К302175,302175
2,К302171,302171
3,К301206,301206
4,К301183,301183
...,...,...
272,9301180,301180
273,9301178,301178
274,9301177,301177
275,9301175,301175


In [9]:
# сделаем выборку первичных проб
idx = results['SampleID'].isin(repeat_codes['Initial SampleID'])
initial_samples = results.loc[idx, :].sort_values(by = 'SampleID', ignore_index = True)
initial_samples


,SampleID,Sr_pct_ПКСА,Ba_pct_ПКСА,TiO2_pct_ПКСА,MnO_pct_ПКСА,V_ppm_ПКСА,Cr_ppm_ПКСА,Co_ppm_ПКСА,Ni_ppm_ПКСА,Be_ppm_ПКСА,Li_ppm_ПКСА,W_ppm_ПКСА,Mo_ppm_ПКСА,Sn_ppm_ПКСА,Cu_ppm_ПКСА,Pb_ppm_ПКСА,Zn_ppm_ПКСА,Cd_ppm_ПКСА,Bi_ppm_ПКСА,Ag_ppm_ПКСА,As_ppm_ПКСА,Sb_ppm_ПКСА
0,301173,0.03,0.05,0.4,0.2,150,200,40,40,0.5,20,1.5,1.2,1,100,20,80,4,0.5,0.06,20,15
1,301175,0.03,0.06,1,0.15,100,200,40,40,0.5,20,1.5,1.2,0.5,120,8,60,4,0.5,0.04,10,15
2,301177,0.15,0.15,1.01,0.25,80,200,50,40,0.5,20,1.5,1.5,0.5,50,6,40,4,0.5,0.04,10,15
3,301178,0.06,0.1,1.01,0.3,60,200,30,20,0.5,5,1.5,3,0.5,40,6,50,4,0.5,0.05,10,15
4,301180,0.03,0.1,1.01,0.15,80,200,30,30,1,10,1.5,1.5,0.5,50,6,50,4,0.5,0.04,10,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,338248,0.02,0.05,0.6,0.04,100,200,15,25,1,20,1.5,1.2,1.2,40,25,50,4,0.5,0.04,10,15
272,338249,0.05,0.15,1.01,0.08,40,200,20,40,3,15,1.5,2.5,0.5,40,12,30,4,0.5,0.04,10,15
273,338271,0.03,0.1,1.01,0.08,60,200,15,30,1.2,25,1.5,3,1.2,30,25,40,4,0.5,0.08,10,15
274,338276,0.025,0.04,0.6,0.06,80,200,10,10,0.5,15,1.5,1.5,1,40,25,60,4,0.5,0.05,10,15


In [11]:
# сделаем выборку повторных анализов
repeat_samples = repeat_codes.merge(results, left_on = 'Repeat SampleID', right_on = 'SampleID', how = 'left')
repeat_samples.sort_values(by = 'Initial SampleID', ignore_index = True, inplace = True)
repeat_samples

,Repeat SampleID,Initial SampleID,SampleID,Sr_pct_ПКСА,Ba_pct_ПКСА,TiO2_pct_ПКСА,MnO_pct_ПКСА,V_ppm_ПКСА,Cr_ppm_ПКСА,Co_ppm_ПКСА,Ni_ppm_ПКСА,Be_ppm_ПКСА,Li_ppm_ПКСА,W_ppm_ПКСА,Mo_ppm_ПКСА,Sn_ppm_ПКСА,Cu_ppm_ПКСА,Pb_ppm_ПКСА,Zn_ppm_ПКСА,Cd_ppm_ПКСА,Bi_ppm_ПКСА,Ag_ppm_ПКСА,As_ppm_ПКСА,Sb_ppm_ПКСА
0,9301173,301173,9301173,0.03,0.05,0.4,0.15,100,120,40,50,0.5,25,1.5,1.5,1,100,25,100,4,0.5,0.06,20,15
1,9301175,301175,9301175,0.03,0.1,1.01,0.1,80,100,40,50,0.5,25,1.5,1.5,1,120,8,60,4,0.5,0.04,10,15
2,9301177,301177,9301177,0.1,0.15,1.01,0.15,60,80,30,25,0.5,15,1.5,1.2,0.5,30,6,30,4,0.5,0.03,10,15
3,9301178,301178,9301178,0.05,0.08,1.01,0.2,60,20,20,8,1.5,5,1.5,3,0.5,30,8,40,4,0.5,0.03,10,15
4,9301180,301180,9301180,0.03,0.12,1.01,0.15,60,100,40,30,0.5,10,1.5,1.2,0.5,40,6,40,4,0.5,0.03,10,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,9338248,338248,9338248,0.02,0.08,0.6,0.05,60,40,15,15,0.5,30,1.5,1.2,1.2,40,25,60,4,0.5,0.03,10,15
272,9338249,338249,9338249,0.05,0.2,1.01,0.08,50,25,20,20,2.5,30,1.5,1.5,1,40,12,40,4,0.5,0.05,10,15
273,К338271,338271,К338271,0.04,0.12,1,0.06,40,40,12,20,1,25,1.5,2.5,1.2,30,20,40,4,0.5,0.1,10,15
274,К338276,338276,К338276,0.02,0.04,0.5,0.06,80,40,10,10,0.5,20,1.5,1.2,1,40,25,60,4,0.5,0.05,10,15


In [15]:
import numpy as np

for col in metadata:
    # короткое имя
    name = metadata.loc['Analyte Symbol', col]
    
    # пропустим столбцы с недостаточной чувствительностью
    if metadata.loc['Sensitivity k', col] >= 0.8:
        print(name, ' пропущен из-за недостаточной чувствительности')
        continue
    
    # сделаем выборку данных
    initial = initial_samples[col].astype(float).copy()
    repeat = repeat_samples[col].astype(float).copy()
    
    # логарифмируем
    initial_lg = np.log(initial)
    repeat_lg = np.log(repeat)
    
    # определяем систематическую ошибку
    d_syst = np.mean(initial_lg - repeat_lg)
    s_syst = np.exp(d_syst)
    
    # введем поправку на систематическую погрешность
    if 0.9 < s_syst < 1.1:
        status = 'Сист. уд.'
    else:
        initial_lg -= d_syst
        status = 'Систе. ИСПРАВ.'
    
    # определяем случайную ошибку
    d_rand = np.mean(np.abs(initial_lg - repeat_lg)) / 2**0.5
    s_rand = np.exp(d_rand)
    
    # статус случайной ошибки
    if s_rand > 1.6:
        rand_status = 'Случ. НЕУД.'
    else:
        rand_status = 'Случ. уд.'
    
    # запишем в метаданные
    metadata.loc['Δ сист.', col] = d_syst
    metadata.loc['δ сист.', col] = s_syst
    metadata.loc['δ случ.', col] = s_rand
    
    print(f"{name}: {s_syst:.2f}, {status}, {s_rand:.2f}, {rand_status}")

Sr: 1.06, Сист. уд., 1.12, Случ. уд.
Ba: 0.96, Сист. уд., 1.17, Случ. уд.
TiO2: 0.97, Сист. уд., 1.15, Случ. уд.
MnO: 1.12, Систе. ИСПРАВ., 1.15, Случ. уд.
V: 1.16, Систе. ИСПРАВ., 1.15, Случ. уд.
Cr: 4.49, Систе. ИСПРАВ., 1.36, Случ. уд.
Co: 0.97, Сист. уд., 1.12, Случ. уд.
Ni: 1.13, Систе. ИСПРАВ., 1.20, Случ. уд.
Be: 1.10, Систе. ИСПРАВ., 1.23, Случ. уд.
Li: 0.81, Систе. ИСПРАВ., 1.19, Случ. уд.
W  пропущен из-за недостаточной чувствительности
Mo: 1.08, Сист. уд., 1.12, Случ. уд.
Sn: 0.92, Сист. уд., 1.18, Случ. уд.
Cu: 1.06, Сист. уд., 1.13, Случ. уд.
Pb: 0.99, Сист. уд., 1.11, Случ. уд.
Zn: 0.99, Сист. уд., 1.12, Случ. уд.
Cd  пропущен из-за недостаточной чувствительности
Bi  пропущен из-за недостаточной чувствительности
Ag: 1.05, Сист. уд., 1.15, Случ. уд.
As  пропущен из-за недостаточной чувствительности
Sb  пропущен из-за недостаточной чувствительности


In [16]:
metadata.to_excel(new_filepath)

,Sr_pct_ПКСА,Ba_pct_ПКСА,TiO2_pct_ПКСА,MnO_pct_ПКСА,V_ppm_ПКСА,Cr_ppm_ПКСА,Co_ppm_ПКСА,Ni_ppm_ПКСА,Be_ppm_ПКСА,Li_ppm_ПКСА,W_ppm_ПКСА,Mo_ppm_ПКСА,Sn_ppm_ПКСА,Cu_ppm_ПКСА,Pb_ppm_ПКСА,Zn_ppm_ПКСА,Cd_ppm_ПКСА,Bi_ppm_ПКСА,Ag_ppm_ПКСА,As_ppm_ПКСА,Sb_ppm_ПКСА
Analyte Symbol,Sr,Ba,TiO2,MnO,V,Cr,Co,Ni,Be,Li,W,Mo,Sn,Cu,Pb,Zn,Cd,Bi,Ag,As,Sb
Unit Symbol,%,%,%,%,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm
Detection Limit,0.01,0.005,0.001,0.001,1,1,1,1,1,10,3,0.5,1,1,1,10,8,1,0.02,20,30
Analysis Method,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА,ПКСА
Sensitivity k,0.026491,0.000154,0.001313,0.000077,0,0,0.00085,0.000077,0.56835,0.200649,0.995907,0.000232,0.285449,0,0.000541,0.010658,0.995984,0.96177,0.026568,0.976908,0.999691
Δ сист.,0.060681,-0.037916,-0.029594,0.115441,0.149572,1.501011,-0.029444,0.125226,0.099349,-0.208087,NaN,0.079604,-0.087689,0.055802,-0.014601,-0.01144,NaN,NaN,0.04866,NaN,NaN
δ сист.,1.062559,0.962794,0.97084,1.122368,1.161337,4.486221,0.970985,1.133405,1.104452,0.812136,NaN,1.082859,0.916046,1.057388,0.985505,0.988625,NaN,NaN,1.049863,NaN,NaN
δ случ.,1.119863,1.167648,1.150884,1.152154,1.14793,1.364945,1.116193,1.195302,1.234024,1.185147,NaN,1.11821,1.181645,1.127219,1.105882,1.12362,NaN,NaN,1.152514,NaN,NaN


In [19]:
col = 'V_ppm_ПКСА'

d_syst = metadata.loc['Δ сист.', col]
idx = results[col].apply(lambda x: isinstance(x, str))
data = results.loc[~idx, col].astype(float)
results.loc[~idx, col] = np.exp(np.log(data) - d_syst)
results[col]

0         86.107621
1         86.107621
2         68.886097
3        129.161431
4         68.886097
            ...    
12963     51.664572
12964     51.664572
12965     51.664572
12966     68.886097
12967     86.107621
Name: V_ppm_ПКСА, Length: 12962, dtype: object